### Тинькофф Образование. Курс Анализ данных в индустрии 2023

Для выполнения задания потребуется самому сгенерировать данные с помощью модуля random (все пункты выполняются с помощью нее).

Что необходимо, для создания будущей таблицы:

0) Для генерации данных в random необходимо установить seed = "АДВИ_2023", всего длинна каждого списка N = 1000;

1) index колонка: создать список из N последовательных целых элементов от 1 до 1000 включительно, далее, с помощью random, перемешать его -> будем использовать как ;

2) groups колонка: создать список из N элементов, состоящий из значений из списка ["группа_1", "группа_2", "группа_3", "группа_4", "группа_5"] (равномерное распределение);

3) uniform колонка: создать список из N элементов, состоящий из целых значение из равномерного распределения [50, 100];

4) gauss_1, gauss_2, gauss_3 колонки: Создать 3 списка из N элементов каждый, заполненные значениями из нормальных распределений с параметрами:  
μ = 0, σ^2 = 1 -> третья колонка gauss_1;  
μ = 0, σ^2 = 144 -> четвертая колонка gauss_2;  
μ = 50, σ^2 = 81 ->  пятая колонка gauss_3;  

5) Создать pandas DataFrame, где (1) будет индексом, (2)-(4) колонки, названия колонок ["groups", "uniform", "gauss_1", "gauss_2", "gauss_3"] соответственно, не забывая проверить, что типы колонок должны соответствовать заявленным типам данных;

6) Для колонок gauss_1 и gauss_2 заменить значения на None:

Для gauss_1_col, если индекс у значение делится на 121 без остатка;  
Для gauss_2_col, если дробная часть больше 0.95;  

7) Теперь, для пропущенных значений gauss_1 и gauss_2:

Для gauss_2_col None заменить на среднее по колонке;  
Удалить строчки из таблицы, где gauss_1_col == None.

In [1]:
import random
import pandas as pd

In [2]:
#устанавливаем seed
random.seed("АДВИ_2023")
#создаем список с индексами
index1 = [i for i in range(1, 1001)]
#перемешиваем индексы
random.shuffle(index1)
#значения для столбца groups
groups1 = ["группа_1", "группа_2", "группа_3", "группа_4", "группа_5"]
#список groups через равномерное распределения
groups1 = random.choices(groups1, k=1000)
#список из 1000 элементов, состоящий из целых значение из равномерного распределения [50, 100] 
uniform1 = [random.randint(50, 100) for _ in range(1000)]
#списки, заполненные значениями из нормальных распределений
gauss_1 = [random.normalvariate(mu = 0, sigma = 1) for i in range(0, 1000)]
gauss_2 = [random.normalvariate(mu = 0, sigma = 12) for i in range(0, 1000)]
gauss_3 = [random.normalvariate(mu = 50, sigma = 9) for i in range(0, 1000)]

In [3]:
#создаем df
df = pd.DataFrame({"groups":groups1,
                   "uniform":uniform1,
                   "gauss_1":gauss_1,
                   "gauss_2":gauss_2,
                   "gauss_3":gauss_3}, index = index1)

In [4]:
#значение делится на 121 без остатка
df.loc[(df.index%121 == 0), 'gauss_1'] = None
#дробная часть больше 0.95
df.loc[(abs(df.gauss_2)%1 > 0.95), 'gauss_2'] = None

In [5]:
#оставить строки, где строки gauss_1 не пустые
df = df[~(df.gauss_1.isna())]
#заполнить пустые средним по колонке
df['gauss_2'] = df['gauss_2'].fillna(df.gauss_2.mean())

### 1. Сколько всего получилось записей в таблице после выполнения всех пунктов из условия?

In [6]:
df.shape[0]

992

### 2. Для колонки gauss_2 посчитать среднее и среднеквадратическое отклонение

In [7]:
#среднее
df.gauss_2.mean()

0.24198967797234985

In [8]:
#среднеквадрат
df.gauss_2.std( )

12.008659263877034

### 3. Какая группа встречается чаще всего и сколько раз?

In [9]:
df.groups.value_counts()

группа_4    222
группа_1    215
группа_2    194
группа_3    184
группа_5    177
Name: groups, dtype: int64

### 4. Для группа_5 вывести самый "ранний" (наименьший) индекс

In [10]:
df[df.groups == 'группа_5'].reset_index().sort_values('index').head(1)

,index,groups,uniform,gauss_1,gauss_2,gauss_3
174,6,группа_5,64,-0.846485,16.232845,54.076235


### 5. Сколько записей из колонки uniform имеют значение не ниже значения 90% перцентиля?

In [11]:
df[df.uniform >= df.uniform.quantile(0.9)].count()

groups     118
uniform    118
gauss_1    118
gauss_2    118
gauss_3    118
dtype: int64

### 6. У какой группы наибольшее значение медианы для gauss_3?

In [12]:
df.groupby('groups')\
  .agg({'gauss_3':'median'})\
  .sort_values('gauss_3', ascending = False)\
  .head(1)

,gauss_3
groups,
группа_4,50.747451


### 7. У какой группы наибольшее минимальное значение gauss_1?

Ответ дать виде группы и значением в gauss_2 (округлить до сотых)

In [13]:
df.groupby('groups')\
  .agg({'gauss_1':'min'})\
  .sort_values('gauss_1', ascending = False)\
  .head(1)

,gauss_1
groups,
группа_2,-2.186968


In [14]:
df.groupby('groups')\
  .agg({'gauss_2':'min'})\
  .sort_values('gauss_2', ascending = False)\
  .query('groups == "группа_2"')

,gauss_2
groups,
группа_2,-30.642554


### 8. Создать новую колонку gauss где значения будут получаться из (gauss_1 + gauss_2) / gauss_3
**Чему будет равен 70% перцентиль (округлить до сотых)?**

In [15]:
df['gauss'] = (df.gauss_1 + df.gauss_2) / df.gauss_3

In [16]:
round(df.gauss.quantile(0.7),2)

0.13